# Importing Libraries

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import os
import glob
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

# Reading of Data

In [3]:
df=pd.read_csv("C:\\Users\\sdsek\\OneDrive\\Desktop\\Kolate.AI\\1183\\EDA\\Data\\1183_Final_Dataset.csv")

In [4]:
df.head()

,Unnamed: 0,SUBJID,LBTEST,LBBASE,LBSTRESN,VISITDY_x,LSCAT,LSNEW,LSSLD,LSLD,LSSITE,LSNEWANY,VISITDY_y,RSNEW,PFSDYCR,PFSCR,AEPT,AECONT,Severity_1.0,Severity_2.0,Severity_3.0,Severity_4.0
0,0,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,-11.0,N,54.0,1.0,Dermatitis acneiform,N,2,0,0,0
1,1,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,55.0,Y,54.0,1.0,Dermatitis acneiform,N,2,0,0,0
2,2,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,55.0,N,54.0,1.0,Dermatitis acneiform,N,2,0,0,0
3,3,3,Albumin,N,36.84,-11.0,Target lesion,N,65.0,35.0,Liver,N,-11.0,N,54.0,1.0,Dermatitis acneiform,N,2,0,0,0
4,4,3,Albumin,N,36.84,-11.0,Target lesion,N,65.0,35.0,Liver,N,55.0,Y,54.0,1.0,Dermatitis acneiform,N,2,0,0,0


In [5]:
df= df.drop(columns=['Unnamed: 0','AEPT', 'AECONT', 'Severity_1.0', 'Severity_2.0','Severity_3.0', 'Severity_4.0'])

In [6]:
df.shape

(7100293, 15)

In [7]:
df.head()

,SUBJID,LBTEST,LBBASE,LBSTRESN,VISITDY_x,LSCAT,LSNEW,LSSLD,LSLD,LSSITE,LSNEWANY,VISITDY_y,RSNEW,PFSDYCR,PFSCR
0,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,-11.0,N,54.0,1.0
1,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,55.0,Y,54.0,1.0
2,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,55.0,N,54.0,1.0
3,3,Albumin,N,36.84,-11.0,Target lesion,N,65.0,35.0,Liver,N,-11.0,N,54.0,1.0
4,3,Albumin,N,36.84,-11.0,Target lesion,N,65.0,35.0,Liver,N,55.0,Y,54.0,1.0


In [8]:
df = df.drop_duplicates(keep='first')

In [9]:
def find_min_max_lssld_excl_zero(subjid_group):
    non_zero_lssld = subjid_group[subjid_group['LSSLD'] != 0]
    if not non_zero_lssld.empty:
        min_visitdy_y = non_zero_lssld['VISITDY_y'].min()
        max_visitdy_y = non_zero_lssld['VISITDY_y'].max()
        min_lssld = non_zero_lssld.loc[non_zero_lssld['VISITDY_y'] == min_visitdy_y, 'LSSLD'].min()
        max_lssld = non_zero_lssld.loc[non_zero_lssld['VISITDY_y'] == max_visitdy_y, 'LSSLD'].max()
        change_in_tumor_size = max_lssld - min_lssld
        return pd.Series({
            'min_VISITDY_y': min_visitdy_y,
            'min_LSSLD': min_lssld,
            'max_VISITDY_y': max_visitdy_y,
            'max_LSSLD': max_lssld,
            'LSSLD_Diff': change_in_tumor_size,
            'Percentage_LSSLD_Change': (change_in_tumor_size / max_lssld) * 100 if min_lssld != 0 else None
        })
    else:
        return pd.Series({
            'min_VISITDY_y': None,
            'min_LSSLD': None,
            'max_VISITDY_y': None,
            'max_LSSLD': None,
            'LSSLD_Diff': None,
            'Percentage_LSSLD_Change': None
        })
results = df.groupby('SUBJID').apply(find_min_max_lssld_excl_zero).reset_index()
print(results)

     SUBJID  min_VISITDY_y  min_LSSLD  max_VISITDY_y  max_LSSLD  LSSLD_Diff  Percentage_LSSLD_Change
0         3          -11.0       65.0           55.0      118.0        53.0                44.915254
1         7          -14.0      136.0          343.0      575.0       439.0                76.347826
2         9           -2.0       30.0          344.0       85.0        55.0                64.705882
3        13           -1.0      166.0           74.0      238.0        72.0                30.252101
4        15          -24.0       49.0          284.0      159.0       110.0                69.182390
5        17           -4.0       39.0          320.0       82.0        43.0                52.439024
6        20           -1.0       89.0           57.0      144.0        55.0                38.194444
7        24           -9.0       11.0          380.0       90.0        79.0                87.777778
8        25          -13.0      103.0          114.0      203.0       100.0                

In [10]:
results['LSSLD_Change'] = np.where(results['Percentage_LSSLD_Change'] > 50, 1, 0)

In [11]:
df=df.merge(results,on='SUBJID',how='inner')

In [12]:
df.shape

(2051290, 22)

In [13]:
df.head()

,SUBJID,LBTEST,LBBASE,LBSTRESN,VISITDY_x,LSCAT,LSNEW,LSSLD,LSLD,LSSITE,LSNEWANY,VISITDY_y,RSNEW,PFSDYCR,PFSCR,min_VISITDY_y,min_LSSLD,max_VISITDY_y,max_LSSLD,LSSLD_Diff,Percentage_LSSLD_Change,LSSLD_Change
0,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,-11.0,N,54.0,1.0,-11.0,65.0,55.0,118.0,53.0,44.915254,0
1,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,55.0,Y,54.0,1.0,-11.0,65.0,55.0,118.0,53.0,44.915254,0
2,3,Albumin,N,36.84,-11.0,Target lesion,N,118.0,39.0,Liver,N,55.0,N,54.0,1.0,-11.0,65.0,55.0,118.0,53.0,44.915254,0
3,3,Albumin,N,36.84,-11.0,Target lesion,N,65.0,35.0,Liver,N,-11.0,N,54.0,1.0,-11.0,65.0,55.0,118.0,53.0,44.915254,0
4,3,Albumin,N,36.84,-11.0,Target lesion,N,65.0,35.0,Liver,N,55.0,Y,54.0,1.0,-11.0,65.0,55.0,118.0,53.0,44.915254,0


In [14]:
numerical_columns = ['LBSTRESN', 'VISITDY_x', 'LSSLD', 'LSLD', 'VISITDY_y', 'PFSDYCR', 'PFSCR']
binary_columns = ['LSNEW', 'LSNEWANY', 'RSNEW','LSCAT','LBBASE','LBTEST','LSSITE']

In [15]:
df_encoded = pd.get_dummies(df, columns=binary_columns, drop_first=True, prefix_sep='_')

In [16]:
reduced_df = df_encoded.groupby('SUBJID').agg(
    {col: 'mean' for col in numerical_columns}
).reset_index()

In [17]:
df.shape

(2051290, 22)

In [18]:
binary_encoded_columns = [col for col in df_encoded.columns if col not in df.columns]
for col in binary_encoded_columns:
    reduced_df[col] = df_encoded.groupby('SUBJID')[col].sum().values

In [19]:
reduced_df = reduced_df[['SUBJID']  + numerical_columns + binary_encoded_columns]

In [20]:
reduced_df.head()

,SUBJID,LBSTRESN,VISITDY_x,LSSLD,LSLD,VISITDY_y,PFSDYCR,PFSCR,LSNEW_Y,LSNEWANY_Y,RSNEW_Y,LSCAT_Non-target lesion,LSCAT_Target lesion,LBBASE_Y,LBTEST_Alkaline Phosphatase,LBTEST_Carcinoembryonic Antigen,LBTEST_Creatinine,LBTEST_Hemoglobin,LBTEST_Lactate Dehydrogenase,LBTEST_Platelets,LBTEST_White Blood Cells,LSSITE_Bone,LSSITE_Central Nervous System,LSSITE_Chest,LSSITE_Gastrointestinal,LSSITE_HEENT/Neck,LSSITE_Liver,LSSITE_Lymph Nodes,LSSITE_Musculoskeletal,LSSITE_Other,LSSITE_Pelvic Site,LSSITE_Pulmonary,LSSITE_Skin,LSSITE_Spleen
0,3,135.910100,26.714286,99.000000,35.727273,33.000000,54.0,1.0,48,288,336,480,480,504,126,126,126,126,126,126,126,0,48,0,0,0,528,144,0,96,0,96,0,0
1,7,153.863690,169.257426,284.521127,40.117647,171.571429,339.0,1.0,0,0,0,1568,9744,5656,1414,1414,1414,1414,1414,1414,1414,0,0,0,0,0,6272,4256,0,0,0,784,0,0
2,9,112.913064,175.000000,44.071429,20.142857,175.000000,301.0,1.0,0,0,0,3136,3136,3136,784,784,784,784,784,784,784,0,0,0,0,0,784,3920,0,0,784,784,0,0
3,13,190.772448,39.909091,206.210526,35.000000,49.000000,73.0,1.0,96,864,528,576,912,792,198,198,198,198,198,198,198,0,0,0,96,0,1008,192,0,0,96,96,0,0
4,15,97.912880,147.196429,93.613636,23.909091,140.500000,414.0,1.0,0,0,0,1152,4224,2688,672,672,672,672,672,672,672,0,0,0,0,0,2304,0,0,0,0,3072,0,0


In [21]:
df1=df[['SUBJID','LSNEWANY','min_VISITDY_y','min_LSSLD','max_VISITDY_y','max_LSSLD','LSSLD_Diff','Percentage_LSSLD_Change','LSSLD_Change']]

In [22]:
df1.head()

,SUBJID,LSNEWANY,min_VISITDY_y,min_LSSLD,max_VISITDY_y,max_LSSLD,LSSLD_Diff,Percentage_LSSLD_Change,LSSLD_Change
0,3,N,-11.0,65.0,55.0,118.0,53.0,44.915254,0
1,3,N,-11.0,65.0,55.0,118.0,53.0,44.915254,0
2,3,N,-11.0,65.0,55.0,118.0,53.0,44.915254,0
3,3,N,-11.0,65.0,55.0,118.0,53.0,44.915254,0
4,3,N,-11.0,65.0,55.0,118.0,53.0,44.915254,0


In [23]:
df1=df1.merge(reduced_df,on='SUBJID',how='inner')

In [24]:
df1 = df1.drop_duplicates(keep='first')

In [25]:
df1.shape

(458, 42)

In [26]:
df1.head()

,SUBJID,LSNEWANY,min_VISITDY_y,min_LSSLD,max_VISITDY_y,max_LSSLD,LSSLD_Diff,Percentage_LSSLD_Change,LSSLD_Change,LBSTRESN,VISITDY_x,LSSLD,LSLD,VISITDY_y,PFSDYCR,PFSCR,LSNEW_Y,LSNEWANY_Y,RSNEW_Y,LSCAT_Non-target lesion,LSCAT_Target lesion,LBBASE_Y,LBTEST_Alkaline Phosphatase,LBTEST_Carcinoembryonic Antigen,LBTEST_Creatinine,LBTEST_Hemoglobin,LBTEST_Lactate Dehydrogenase,LBTEST_Platelets,LBTEST_White Blood Cells,LSSITE_Bone,LSSITE_Central Nervous System,LSSITE_Chest,LSSITE_Gastrointestinal,LSSITE_HEENT/Neck,LSSITE_Liver,LSSITE_Lymph Nodes,LSSITE_Musculoskeletal,LSSITE_Other,LSSITE_Pelvic Site,LSSITE_Pulmonary,LSSITE_Skin,LSSITE_Spleen
0,3,N,-11.0,65.0,55.0,118.0,53.0,44.915254,0,135.910100,26.714286,99.000000,35.727273,33.000000,54.0,1.0,48,288,336,480,480,504,126,126,126,126,126,126,126,0,48,0,0,0,528,144,0,96,0,96,0,0
27,3,Y,-11.0,65.0,55.0,118.0,53.0,44.915254,0,135.910100,26.714286,99.000000,35.727273,33.000000,54.0,1.0,48,288,336,480,480,504,126,126,126,126,126,126,126,0,48,0,0,0,528,144,0,96,0,96,0,0
1008,7,N,-14.0,136.0,343.0,575.0,439.0,76.347826,1,153.863690,169.257426,284.521127,40.117647,171.571429,339.0,1.0,0,0,0,1568,9744,5656,1414,1414,1414,1414,1414,1414,1414,0,0,0,0,0,6272,4256,0,0,0,784,0,0
12320,9,N,-2.0,30.0,344.0,85.0,55.0,64.705882,1,112.913064,175.000000,44.071429,20.142857,175.000000,301.0,1.0,0,0,0,3136,3136,3136,784,784,784,784,784,784,784,0,0,0,0,0,784,3920,0,0,784,784,0,0
18592,13,N,-1.0,166.0,74.0,238.0,72.0,30.252101,0,190.772448,39.909091,206.210526,35.000000,49.000000,73.0,1.0,96,864,528,576,912,792,198,198,198,198,198,198,198,0,0,0,96,0,1008,192,0,0,96,96,0,0


In [79]:
result1=df1

In [80]:
result1.fillna(0, inplace=True)

In [81]:
result1['LSNEWANY'] = result1['LSNEWANY'].astype('category').cat.codes

In [82]:
result1.head()

,SUBJID,LSNEWANY,min_VISITDY_y,min_LSSLD,max_VISITDY_y,max_LSSLD,LSSLD_Diff,Percentage_LSSLD_Change,LSSLD_Change,LBSTRESN,VISITDY_x,LSSLD,LSLD,VISITDY_y,PFSDYCR,PFSCR,LSNEW_Y,LSNEWANY_Y,RSNEW_Y,LSCAT_Non-target lesion,LSCAT_Target lesion,LBBASE_Y,LBTEST_Alkaline Phosphatase,LBTEST_Carcinoembryonic Antigen,LBTEST_Creatinine,LBTEST_Hemoglobin,LBTEST_Lactate Dehydrogenase,LBTEST_Platelets,LBTEST_White Blood Cells,LSSITE_Bone,LSSITE_Central Nervous System,LSSITE_Chest,LSSITE_Gastrointestinal,LSSITE_HEENT/Neck,LSSITE_Liver,LSSITE_Lymph Nodes,LSSITE_Musculoskeletal,LSSITE_Other,LSSITE_Pelvic Site,LSSITE_Pulmonary,LSSITE_Skin,LSSITE_Spleen,Predicted_New_Lesions
0,3,0,-11.0,65.0,55.0,118.0,53.0,44.915254,0,135.910100,26.714286,99.000000,35.727273,33.000000,54.0,1.0,48,288,336,480,480,504,126,126,126,126,126,126,126,0,48,0,0,0,528,144,0,96,0,96,0,0,0.334397
27,3,1,-11.0,65.0,55.0,118.0,53.0,44.915254,0,135.910100,26.714286,99.000000,35.727273,33.000000,54.0,1.0,48,288,336,480,480,504,126,126,126,126,126,126,126,0,48,0,0,0,528,144,0,96,0,96,0,0,0.334397
1008,7,0,-14.0,136.0,343.0,575.0,439.0,76.347826,1,153.863690,169.257426,284.521127,40.117647,171.571429,339.0,1.0,0,0,0,1568,9744,5656,1414,1414,1414,1414,1414,1414,1414,0,0,0,0,0,6272,4256,0,0,0,784,0,0,0.252835
12320,9,0,-2.0,30.0,344.0,85.0,55.0,64.705882,1,112.913064,175.000000,44.071429,20.142857,175.000000,301.0,1.0,0,0,0,3136,3136,3136,784,784,784,784,784,784,784,0,0,0,0,0,784,3920,0,0,784,784,0,0,0.339206
18592,13,0,-1.0,166.0,74.0,238.0,72.0,30.252101,0,190.772448,39.909091,206.210526,35.000000,49.000000,73.0,1.0,96,864,528,576,912,792,198,198,198,198,198,198,198,0,0,0,96,0,1008,192,0,0,96,96,0,0,0.361022


#  Regression for New Lesions

# Linear Regression for New Lesions

In [90]:
X = result1.drop(columns=['SUBJID', 'LSNEWANY'])  
y = result1['LSNEWANY']

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [93]:
model11 = LinearRegression()

In [94]:
model11.fit(X_train_scaled, y_train)

LinearRegression()

In [95]:
y_test_pred = model11.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.2461325303908117


In [96]:
print(f"Intercept: {model11.intercept_}")
print(f"Coefficients: {model11.coef_}")

Intercept: 0.2896174863387972
Coefficients: [   0.54230937    2.64582735  -16.67207937    1.87072002    0.45108579
    1.48966443   -3.9136074    -0.61091038    2.60374139   -4.79980771
   -1.64944601   24.58933701   -5.76370745    3.21509547   -3.00394199
   -1.04042887   20.96988583  -29.76303049  -27.28632399   39.61501445
  -27.93729628   -3.49306632   42.58486277   42.58486277 -134.45591521
   52.4502462    42.58486277   -0.45441626    0.2112928     0.38011977
   -4.88138924    1.61352352   -6.15834706    0.37924131  -17.76469951
    0.45821322    2.88497021   -0.21378094   -3.51693239   -0.36743794
  -11.75321521]


In [73]:
print(f"Feature names: {X.columns.tolist()}")

Feature names: ['min_VISITDY_y', 'min_LSSLD', 'max_VISITDY_y', 'max_LSSLD', 'LSSLD_Diff', 'Percentage_LSSLD_Change', 'LSSLD_Change', 'LBSTRESN', 'VISITDY_x', 'LSSLD', 'LSLD', 'VISITDY_y', 'PFSDYCR', 'PFSCR', 'LSNEW_Y', 'LSNEWANY_Y', 'RSNEW_Y', 'LSCAT_Non-target lesion', 'LSCAT_Target lesion', 'LBBASE_Y', 'LBTEST_Alkaline Phosphatase', 'LBTEST_Carcinoembryonic Antigen', 'LBTEST_Creatinine', 'LBTEST_Hemoglobin', 'LBTEST_Lactate Dehydrogenase', 'LBTEST_Platelets', 'LBTEST_White Blood Cells', 'LSSITE_Bone', 'LSSITE_Central Nervous System', 'LSSITE_Chest', 'LSSITE_Gastrointestinal', 'LSSITE_HEENT/Neck', 'LSSITE_Liver', 'LSSITE_Lymph Nodes', 'LSSITE_Musculoskeletal', 'LSSITE_Other', 'LSSITE_Pelvic Site', 'LSSITE_Pulmonary', 'LSSITE_Skin', 'LSSITE_Spleen', 'Predicted_New_Lesions']


In [97]:
predictions_df = pd.DataFrame({
    'SUBJID': result1.loc[X_test.index, 'SUBJID'],  # Extract SUBJID from original dataset
    'LSNEWANY': y_test,
    'Predicted_New_Lesions': y_test_pred
})

In [98]:
predictions_df.head()

,SUBJID,LSNEWANY,Predicted_New_Lesions
1414583,799,0,-0.097855
163703,101,1,0.374826
1583842,888,0,0.033875
1026957,588,1,0.355733
746533,446,0,0.692859


# Logistic Regression for New Lesions

In [99]:
X = result1.drop(columns=['SUBJID', 'LSNEWANY'])  
y = result1['LSNEWANY']

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [101]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [102]:
model12= LogisticRegression()

In [103]:
model12.fit(X_train_scaled, y_train)

LogisticRegression()

In [104]:
y_pred = model12.predict(X_test_scaled)

In [105]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6848


In [106]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[56 12]
 [17  7]]


In [107]:
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.82      0.79        68
           1       0.37      0.29      0.33        24

    accuracy                           0.68        92
   macro avg       0.57      0.56      0.56        92
weighted avg       0.66      0.68      0.67        92



In [113]:
result1['Predicted_New_Lesions'] = model12.predict(scaler.transform(X))

# Final output: One prediction per SUBJID
predictions = result1.drop_duplicates(subset='SUBJID')[['SUBJID', 'LSNEWANY', 'Predicted_New_Lesions']]
predictions.drop_duplicates(keep='first')

,SUBJID,LSNEWANY,Predicted_New_Lesions
0,3,0,0
1008,7,0,0
12320,9,0,0
18592,13,0,0
20176,15,0,0
25552,17,0,0
28720,20,0,0
29168,24,0,0
30688,25,0,0
32263,27,0,0


In [39]:
result1['SUBJID'].nunique()

335

In [40]:
predictions.head()

,SUBJID,LSNEWANY,Predicted_New_Lesions
0,3,0,0
1008,7,0,0
12320,9,0,0
18592,13,0,0
20176,15,0,0


# Overfitting for Logistics New Lesions

In [114]:
y_train_pred = model12.predict(X_train_scaled)

y_test_pred = model12.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

Training Accuracy: 0.7350
Test Accuracy: 0.6848

Potential Overfitting Detected!


# Selecting Features for Logistic New Lesions

In [115]:
result1.columns

Index(['SUBJID', 'LSNEWANY', 'min_VISITDY_y', 'min_LSSLD', 'max_VISITDY_y', 'max_LSSLD', 'LSSLD_Diff', 'Percentage_LSSLD_Change', 'LSSLD_Change', 'LBSTRESN', 'VISITDY_x', 'LSSLD', 'LSLD', 'VISITDY_y', 'PFSDYCR', 'PFSCR', 'LSNEW_Y', 'LSNEWANY_Y', 'RSNEW_Y', 'LSCAT_Non-target lesion', 'LSCAT_Target lesion', 'LBBASE_Y', 'LBTEST_Alkaline Phosphatase', 'LBTEST_Carcinoembryonic Antigen', 'LBTEST_Creatinine', 'LBTEST_Hemoglobin', 'LBTEST_Lactate Dehydrogenase', 'LBTEST_Platelets', 'LBTEST_White Blood Cells', 'LSSITE_Bone', 'LSSITE_Central Nervous System', 'LSSITE_Chest', 'LSSITE_Gastrointestinal', 'LSSITE_HEENT/Neck', 'LSSITE_Liver', 'LSSITE_Lymph Nodes', 'LSSITE_Musculoskeletal', 'LSSITE_Other', 'LSSITE_Pelvic Site', 'LSSITE_Pulmonary', 'LSSITE_Skin', 'LSSITE_Spleen', 'Predicted_New_Lesions'], dtype='object')

In [124]:
X = result1.drop(columns=['SUBJID', 'LSNEWANY','LSNEW_Y','LSNEWANY_Y','LSSITE_Bone', 'LSSITE_Central Nervous System', 'LSSITE_Chest', 'LSSITE_Gastrointestinal', 'LSSITE_HEENT/Neck', 'LSSITE_Liver', 'LSSITE_Lymph Nodes', 'LSSITE_Musculoskeletal', 'LSSITE_Other', 'LSSITE_Pelvic Site', 'LSSITE_Pulmonary', 'LSSITE_Skin', 'LSSITE_Spleen', 'Predicted_New_Lesions'])  
y = result1['LSNEWANY']

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [126]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [127]:
model13 = LogisticRegression()

In [128]:
model13.fit(X_train_scaled, y_train)

LogisticRegression()

In [129]:
y_train_pred = model13.predict(X_train_scaled)
y_test_pred = model13.predict(X_test_scaled)

In [130]:
# Evaluate model performance
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [131]:
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Compare training and test accuracy
if train_accuracy > test_accuracy:
    print("Model may be overfitting to the training data.")
else:
    print("Model is performing similarly on both training and test data, which is good.")

Training Accuracy: 0.7377
Test Accuracy: 0.7065
Model may be overfitting to the training data.


# Removing Overfitting by Cross Validation

In [132]:
X = result1.drop(columns=['SUBJID', 'LSNEWANY','LSNEW_Y','LSNEWANY_Y'])  
y = result1['LSNEWANY']

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [134]:
model14 = LogisticRegression()

In [135]:
cv_scores = cross_val_score(model14, X_train_scaled, y_train, cv=10)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

Cross-validation scores: [0.72972973 0.59459459 0.64864865 0.62162162 0.7027027  0.7027027
 0.75       0.77777778 0.72222222 0.69444444]
Mean CV accuracy: 0.6944


In [136]:
model14.fit(X_train_scaled, y_train)

LogisticRegression()

In [137]:
y_test_pred = model14.predict(X_test_scaled)

In [138]:
# Evaluate the model on test data
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7065


In [139]:
print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_test_pred))

print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))


Confusion Matrix on Test Data:
[[63  5]
 [22  2]]

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.74      0.93      0.82        68
           1       0.29      0.08      0.13        24

    accuracy                           0.71        92
   macro avg       0.51      0.50      0.48        92
weighted avg       0.62      0.71      0.64        92



In [141]:
y_train_pred = model14.predict(X_train_scaled)

y_test_pred = model14.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

Training Accuracy: 0.7377
Test Accuracy: 0.7065

Potential Overfitting Detected!


# XGB Boost for New Lesions

pip install xgboost

In [164]:
import xgboost as xgb

In [165]:
result1= result1.drop(columns=['Predicted_New_Lesions'])

In [166]:
features = result1.drop(columns=['SUBJID', 'LSNEWANY'])  
target = result1['LSNEWANY']

In [167]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, test_size=0.2, random_state=42)

In [169]:
# Initialize and fit the XGBoost model
model18 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model18.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [170]:
# Predict on the test set
y_pred = model18.predict(X_test)

In [171]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [172]:
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.5000
Confusion Matrix:
[[39 29]
 [17  7]]
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.57      0.63        68
           1       0.19      0.29      0.23        24

    accuracy                           0.50        92
   macro avg       0.45      0.43      0.43        92
weighted avg       0.57      0.50      0.53        92



In [174]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train_pred = model18.predict(X_train_scaled)

y_test_pred = model18.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

Training Accuracy: 0.7869
Test Accuracy: 0.5109

Potential Overfitting Detected!


In [175]:
# Make predictions on the full dataset
full_predictions = model18.predict(X_scaled)

In [176]:
# Create DataFrame with predictions
result1['Predicted_New_Lesions'] = full_predictions

In [177]:
# Keep only the first prediction for each SUBJID
unique_predictions_df = result1.drop_duplicates(subset='SUBJID')[['SUBJID', 'LSNEWANY', 'Predicted_New_Lesions']]

In [178]:
print(unique_predictions_df)

         SUBJID  LSNEWANY  Predicted_New_Lesions
0             3         0                      1
1008          7         0                      0
12320         9         0                      0
18592        13         0                      0
20176        15         0                      0
25552        17         0                      0
28720        20         0                      0
29168        24         0                      0
30688        25         0                      0
32263        27         0                      0
32519        31         0                      0
46439        35         0                      1
48103        37         0                      1
78583        39         0                      0
81303        40         0                      0
86103        41         0                      0
92007        42         0                      0
96903        53         0                      0
101063       63         0                      0
103111       67     

# Hyperparameter Tuning and Regularization

In [216]:
X = result1.drop(columns=['SUBJID', 'LSNEWANY','LSNEW_Y','LSNEWANY_Y'])  
y = result1['LSNEWANY']

In [260]:
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X = result1.drop(columns=['SUBJID', 'LSNEWANY','LSNEW_Y','LSNEWANY_Y','Predicted_New_Lesions','min_VISITDY_y','min_LSSLD','max_VISITDY_y','max_LSSLD','LSSLD_Diff','LBTEST_Alkaline Phosphatase','LBTEST_Carcinoembryonic Antigen','LBTEST_Creatinine','LBTEST_Hemoglobin','LBTEST_Lactate Dehydrogenase','LBTEST_Platelets','LBTEST_White Blood Cells','Percentage_LSSLD_Change','LSCAT_Non-target lesion','LSCAT_Target lesion','LBBASE_Y','RSNEW_Y','LBBASE_Y','VISITDY_x'])  
y = result1['LSNEWANY']

In [261]:
X

,LSSLD_Change,LBSTRESN,LSSLD,LSLD,VISITDY_y,PFSDYCR,PFSCR,LSSITE_Bone,LSSITE_Central Nervous System,LSSITE_Chest,LSSITE_Gastrointestinal,LSSITE_HEENT/Neck,LSSITE_Liver,LSSITE_Lymph Nodes,LSSITE_Musculoskeletal,LSSITE_Other,LSSITE_Pelvic Site,LSSITE_Pulmonary,LSSITE_Skin,LSSITE_Spleen
0,0,135.910100,99.000000,35.727273,33.000000,54.0,1.0,0,48,0,0,0,528,144,0,96,0,96,0,0
27,0,135.910100,99.000000,35.727273,33.000000,54.0,1.0,0,48,0,0,0,528,144,0,96,0,96,0,0
1008,1,153.863690,284.521127,40.117647,171.571429,339.0,1.0,0,0,0,0,0,6272,4256,0,0,0,784,0,0
12320,1,112.913064,44.071429,20.142857,175.000000,301.0,1.0,0,0,0,0,0,784,3920,0,0,784,784,0,0
18592,0,190.772448,206.210526,35.000000,49.000000,73.0,1.0,0,0,0,96,0,1008,192,0,0,96,96,0,0
18637,0,190.772448,206.210526,35.000000,49.000000,73.0,1.0,0,0,0,96,0,1008,192,0,0,96,96,0,0
20176,1,97.912880,93.613636,23.909091,140.500000,414.0,1.0,0,0,0,0,0,2304,0,0,0,0,3072,0,0
25552,1,88.209722,46.190476,23.095238,147.333333,226.0,0.0,0,0,0,0,0,2592,576,0,0,0,0,0,0
28720,0,121.011607,117.600000,47.600000,28.000000,56.0,1.0,0,0,0,0,0,384,64,0,0,0,0,0,0
29168,1,121.885268,54.153846,30.923077,166.600000,306.0,1.0,0,0,0,0,0,1520,0,0,0,0,0,0,0


In [262]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model15 = LogisticRegression()

In [263]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization parameter
    'penalty': ['l1', 'l2']  # Penalty term ('l1' for Lasso, 'l2' for Ridge)
}

In [264]:
grid_search = GridSearchCV(estimator=model15, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_

In [265]:
best_model.fit(X_train_scaled, y_train)

LogisticRegression(C=0.1)

In [266]:
y_test_pred = best_model.predict(X_test_scaled)

# Evaluate the best model on test data
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_test_pred))

print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))


Test Accuracy: 0.8700

Confusion Matrix on Test Data:
[[87  6]
 [20 87]]

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.81      0.94      0.87        93
           1       0.94      0.81      0.87       107

    accuracy                           0.87       200
   macro avg       0.87      0.87      0.87       200
weighted avg       0.88      0.87      0.87       200



In [267]:
print("\nBest Hyperparameters:")
print(grid_search.best_params_)


Best Hyperparameters:
{'C': 0.1, 'penalty': 'l2'}


In [268]:
y_train_pred = best_model.predict(X_train_scaled)

y_test_pred = best_model.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")


Training Accuracy: 0.8812
Test Accuracy: 0.8700

Potential Overfitting Detected!


In [274]:
result1['Predicted_New_Lesions'] = best_model.predict(scaler.transform(X))

# Sort data by SUBJID and drop duplicates, keeping the first occurrence
final_predictions = result1.sort_values(by='SUBJID').drop_duplicates(subset='SUBJID', keep='first')

# Select relevant columns for the final output
final_predictions = final_predictions[['SUBJID', 'LSNEWANY', 'Predicted_New_Lesions']]

# Save the final predictions to a CSV file
final_predictions.to_csv("C:\\Users\\sdsek\\OneDrive\\Desktop\\Kolate.AI\\1183\\Regression Model (Tumour Size,New Lesions)\\Output\\Predictions for New Lesions.csv", index=False)
print("Final predictions saved to 'final_predictions.csv'")

Final predictions saved to 'final_predictions.csv'


In [275]:
final_predictions.tail(10)

,SUBJID,LSNEWANY,Predicted_New_Lesions
2021821,1144,0,1
2024621,1146,0,0
2024666,1147,0,1
2027066,1155,0,1
2028090,1156,0,1
2029770,1163,0,1
2040629,1164,1,1
2046810,1165,0,0
2047514,1172,0,1
2049498,1181,0,0


# Removing Overfitting by Elastic Net

In [152]:
X = result1.drop(columns=['SUBJID', 'LSNEWANY','LSNEW_Y','LSNEWANY_Y'])  
y = result1['LSNEWANY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
param_grid = {
    'penalty': ['elasticnet'],
    'solver': ['saga'],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9],  # L1 ratio for elastic net (0 means Ridge, 1 means Lasso)
    'C': [0.1, 1.0, 10.0]  # Regularization parameter
}

In [153]:
model16 = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=10, scoring='accuracy')

In [154]:
model16.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.1, 1.0, 10.0],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9],
                         'penalty': ['elasticnet'], 'solver': ['saga']},
             scoring='accuracy')

In [155]:
best_model = model16.best_estimator_
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)

In [156]:
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Compare training and test accuracy
if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

Training Accuracy: 0.7404
Test Accuracy: 0.6848

Potential Overfitting Detected!


In [158]:
print("\nBest Parameters:")
print(model16.best_params_)


Best Parameters:
{'C': 0.1, 'l1_ratio': 0.7, 'penalty': 'elasticnet', 'solver': 'saga'}


In [159]:
# Cross-validation to further evaluate model performance
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5)
print(f"\nCross-validation scores: {cv_scores}")
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")


Cross-validation scores: [0.7027027  0.67123288 0.71232877 0.73972603 0.69863014]
Mean CV accuracy: 0.7049 (+/- 0.0221)


In [160]:
# Other metrics
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_test_pred))


Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.75      0.87      0.80        68
           1       0.31      0.17      0.22        24

    accuracy                           0.68        92
   macro avg       0.53      0.52      0.51        92
weighted avg       0.63      0.68      0.65        92


Confusion Matrix on Test Data:
[[59  9]
 [20  4]]


# Using SMOTE

In [162]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter

X = result1.drop(columns=['SUBJID', 'LSNEWANY','LSNEW_Y','LSNEWANY_Y','LSSITE_Bone', 'LSSITE_Central Nervous System', 'LSSITE_Chest', 'LSSITE_Gastrointestinal', 'LSSITE_HEENT/Neck', 'LSSITE_Liver', 'LSSITE_Lymph Nodes', 'LSSITE_Musculoskeletal', 'LSSITE_Other', 'LSSITE_Pelvic Site', 'LSSITE_Pulmonary', 'LSSITE_Skin', 'LSSITE_Spleen'])  
y = result1['LSNEWANY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train_scaled, y_train)

print(f"Original dataset shape: {Counter(y_train)}")
print(f"Resampled dataset shape: {Counter(y_res)}")

# Define XGBoost model
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

model17 = GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring='accuracy')
model17.fit(X_res, y_res)

# Get the best model from GridSearchCV
best_model = model17.best_estimator_

# Predict on the original test data
y_test_pred = best_model.predict(X_test_scaled)

# Evaluate model performance
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Classification report and confusion matrix
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_test_pred))

# Save the predictions for each SUBJID
predictions = pd.DataFrame({
    'SUBJID': X_test.index,  # Replace with actual index or identifier for SUBJID
    'LSNEWANY': y_test,
    'Predicted_New_Lesions': y_test_pred
})

predictions.to_csv('predictions.csv', index=False)
print("\nPredictions saved to predictions.csv")


Original dataset shape: Counter({0: 260, 1: 106})
Resampled dataset shape: Counter({0: 260, 1: 260})
Test Accuracy: 0.6848

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       1.00      0.57      0.73        68
           1       0.45      1.00      0.62        24

    accuracy                           0.68        92
   macro avg       0.73      0.79      0.68        92
weighted avg       0.86      0.68      0.70        92


Confusion Matrix on Test Data:
[[39 29]
 [ 0 24]]

Predictions saved to predictions.csv


In [163]:
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Compare training and test accuracy
if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

Training Accuracy: 0.7404
Test Accuracy: 0.6848

Potential Overfitting Detected!


In [175]:
predictions.shape

(92, 3)

# Logistic Regression for Increase in Tumour Size

In [283]:
result1.head()

,SUBJID,LSNEWANY,min_VISITDY_y,min_LSSLD,max_VISITDY_y,max_LSSLD,LSSLD_Diff,Percentage_LSSLD_Change,LSSLD_Change,LBSTRESN,VISITDY_x,LSSLD,LSLD,VISITDY_y,PFSDYCR,PFSCR,LSNEW_Y,LSNEWANY_Y,RSNEW_Y,LSCAT_Non-target lesion,LSCAT_Target lesion,LBBASE_Y,LBTEST_Alkaline Phosphatase,LBTEST_Carcinoembryonic Antigen,LBTEST_Creatinine,LBTEST_Hemoglobin,LBTEST_Lactate Dehydrogenase,LBTEST_Platelets,LBTEST_White Blood Cells,LSSITE_Bone,LSSITE_Central Nervous System,LSSITE_Chest,LSSITE_Gastrointestinal,LSSITE_HEENT/Neck,LSSITE_Liver,LSSITE_Lymph Nodes,LSSITE_Musculoskeletal,LSSITE_Other,LSSITE_Pelvic Site,LSSITE_Pulmonary,LSSITE_Skin,LSSITE_Spleen
0,3,0,-11.0,65.0,55.0,118.0,53.0,44.915254,0,135.910100,26.714286,99.000000,35.727273,33.000000,54.0,1.0,48,288,336,480,480,504,126,126,126,126,126,126,126,0,48,0,0,0,528,144,0,96,0,96,0,0
27,3,1,-11.0,65.0,55.0,118.0,53.0,44.915254,0,135.910100,26.714286,99.000000,35.727273,33.000000,54.0,1.0,48,288,336,480,480,504,126,126,126,126,126,126,126,0,48,0,0,0,528,144,0,96,0,96,0,0
1008,7,0,-14.0,136.0,343.0,575.0,439.0,76.347826,1,153.863690,169.257426,284.521127,40.117647,171.571429,339.0,1.0,0,0,0,1568,9744,5656,1414,1414,1414,1414,1414,1414,1414,0,0,0,0,0,6272,4256,0,0,0,784,0,0
12320,9,0,-2.0,30.0,344.0,85.0,55.0,64.705882,1,112.913064,175.000000,44.071429,20.142857,175.000000,301.0,1.0,0,0,0,3136,3136,3136,784,784,784,784,784,784,784,0,0,0,0,0,784,3920,0,0,784,784,0,0
18592,13,0,-1.0,166.0,74.0,238.0,72.0,30.252101,0,190.772448,39.909091,206.210526,35.000000,49.000000,73.0,1.0,96,864,528,576,912,792,198,198,198,198,198,198,198,0,0,0,96,0,1008,192,0,0,96,96,0,0


In [285]:
#result1= result1.drop(columns=['Predicted_New_Lesions'])

In [286]:
X = result1.drop(columns=['SUBJID', 'LSSLD_Change'])  
y = result1['LSSLD_Change']

In [287]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [288]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [289]:
model21 = LogisticRegression()

In [290]:
model21.fit(X_train_scaled, y_train)

LogisticRegression()

In [291]:
y_pred = model21.predict(X_test_scaled)

In [292]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9565


In [293]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[29  3]
 [ 1 59]]


In [294]:
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94        32
           1       0.95      0.98      0.97        60

    accuracy                           0.96        92
   macro avg       0.96      0.94      0.95        92
weighted avg       0.96      0.96      0.96        92



In [295]:
result1['Predicted_LSSLD_Change'] = model21.predict(scaler.transform(X))

predictions1 = result1.drop_duplicates(subset='SUBJID')[['SUBJID', 'LSSLD_Change', 'Predicted_LSSLD_Change']]

In [296]:
predictions1.head(20)

,SUBJID,LSSLD_Change,Predicted_LSSLD_Change
0,3,0,0
1008,7,1,1
12320,9,1,1
18592,13,0,0
20176,15,1,1
25552,17,1,1
28720,20,0,0
29168,24,1,1
30688,25,0,0
32263,27,0,0


# Checking for Overfitting

In [299]:
y_train_pred = model21.predict(X_train_scaled)

y_test_pred = model21.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Training Accuracy: 0.9863
Test Accuracy: 0.9565


In [301]:
# Check for overfitting
if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")


Potential Overfitting Detected!


# Removing Overfitting by Cross Validation

In [302]:
X = result1.drop(columns=['SUBJID', 'LSSLD_Change'])  
y = result1['LSSLD_Change']
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

In [303]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [304]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [305]:
# Initialize the model with regularization (L2 regularization by default)
model22 = LogisticRegression()

In [306]:
cv_scores = cross_val_score(model22, X_train_scaled, y_train, cv=5)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

Cross-validation scores: [0.85625 0.875   0.85625 0.86875 0.88125]
Mean CV accuracy: 0.8675


In [307]:
# Fit the model on the training data
model22.fit(X_train_scaled, y_train)

LogisticRegression()

In [308]:
# Predict on the test data
y_test_pred = model22.predict(X_test_scaled)

In [309]:
# Evaluate the model on test data
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.8550


In [310]:
print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_test_pred))

print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))


Confusion Matrix on Test Data:
[[85  8]
 [21 86]]

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.80      0.91      0.85        93
           1       0.91      0.80      0.86       107

    accuracy                           0.85       200
   macro avg       0.86      0.86      0.85       200
weighted avg       0.86      0.85      0.86       200



In [312]:
y_train_pred = model22.predict(X_train_scaled)

y_test_pred = model22.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

Training Accuracy: 0.8825
Test Accuracy: 0.8550

Potential Overfitting Detected!


# Regularization

In [313]:
X = result1.drop(columns=['SUBJID', 'LSSLD_Change'])  
y = result1['LSSLD_Change']
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model23 = LogisticRegression()

In [314]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization parameter
    'penalty': ['l1', 'l2']  # Penalty term ('l1' for Lasso, 'l2' for Ridge)
}

grid_search = GridSearchCV(estimator=model23, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [315]:
# Get the best model from grid search
best_model = grid_search.best_estimator_

# Fit the best model on the training data
best_model.fit(X_train_scaled, y_train)

LogisticRegression(C=0.1)

In [316]:
# Predict on the test data
y_test_pred = best_model.predict(X_test_scaled)

# Evaluate the best model on test data
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_test_pred))

print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

Test Accuracy: 0.8700

Confusion Matrix on Test Data:
[[87  6]
 [20 87]]

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.81      0.94      0.87        93
           1       0.94      0.81      0.87       107

    accuracy                           0.87       200
   macro avg       0.87      0.87      0.87       200
weighted avg       0.88      0.87      0.87       200



In [317]:
y_train_pred = best_model.predict(X_train_scaled)

y_test_pred = best_model.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

Training Accuracy: 0.8812
Test Accuracy: 0.8700

Potential Overfitting Detected!


# Solving Overfitting using Elastic Net

In [318]:
X = result1.drop(columns=['SUBJID', 'LSSLD_Change'])  
y = result1['LSSLD_Change']

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [320]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [321]:
param_grid = {
    'penalty': ['elasticnet'],
    'solver': ['saga'],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9],  # L1 ratio for elastic net (0 means Ridge, 1 means Lasso)
    'C': [0.1, 1.0, 10.0]  # Regularization parameter
}

In [322]:
model24= GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')

In [323]:
model24.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.1, 1.0, 10.0],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9],
                         'penalty': ['elasticnet'], 'solver': ['saga']},
             scoring='accuracy')

In [325]:
best_model = model24.best_estimator_

In [326]:
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [330]:
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Compare training and test accuracy
if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")

# Print best parameters found by GridSearchCV
print("\nBest Parameters:")
print(model24.best_params_)

if train_accuracy > test_accuracy:
    print("\nPotential Overfitting Detected!")
else:
    print("\nNo Overfitting Detected!")


Training Accuracy: 0.9863
Test Accuracy: 0.9565

Potential Overfitting Detected!

Best Parameters:
{'C': 0.1, 'l1_ratio': 0.1, 'penalty': 'elasticnet', 'solver': 'saga'}

Potential Overfitting Detected!


In [329]:
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5)
print(f"\nCross-validation scores: {cv_scores}")
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

# Other metrics
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_test_pred))


Cross-validation scores: [1.         0.97260274 0.98630137 1.         0.97260274]
Mean CV accuracy: 0.9863 (+/- 0.0123)

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94        32
           1       0.95      0.98      0.97        60

    accuracy                           0.96        92
   macro avg       0.96      0.94      0.95        92
weighted avg       0.96      0.96      0.96        92


Confusion Matrix on Test Data:
[[29  3]
 [ 1 59]]


# Changing the features choosen

In [331]:
X = result1[['LBSTRESN','VISITDY_x','LSSLD','LSLD','VISITDY_y','PFSDYCR','PFSCR','LSNEW_Y','LSNEWANY_Y','RSNEW_Y','LSCAT_Non-target lesion','LSCAT_Target lesion','LBBASE_Y','LBTEST_Alkaline Phosphatase','LBTEST_Carcinoembryonic Antigen','LBTEST_Creatinine','LBTEST_Hemoglobin','LBTEST_Lactate Dehydrogenase','LBTEST_Platelets','LBTEST_White Blood Cells']]
y = result1['LSSLD_Change']

In [332]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [333]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [334]:
model25 = LogisticRegression()

In [335]:
model25.fit(X_train_scaled, y_train)

LogisticRegression()

In [336]:
y_train_pred = model25.predict(X_train_scaled)
y_test_pred = model25.predict(X_test_scaled)

# Evaluate model performance
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Compare training and test accuracy
if train_accuracy > test_accuracy:
    print("Model may be overfitting to the training data.")
else:
    print("Model is performing similarly on both training and test data, which is good.")


Training Accuracy: 0.8087
Test Accuracy: 0.8261
Model is performing similarly on both training and test data, which is good.


# Therefore we can conclude that by removing LSSITE we Overfitting problem is solved.

In [338]:
y_pred = model25.predict(X_test_scaled)

In [339]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8261


In [340]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[24  8]
 [ 8 52]]


In [341]:
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        32
           1       0.87      0.87      0.87        60

    accuracy                           0.83        92
   macro avg       0.81      0.81      0.81        92
weighted avg       0.83      0.83      0.83        92



In [342]:
result1['Predicted_LSSLD_Change'] = model25.predict(scaler.transform(X))

predictions1 = result1.drop_duplicates(subset='SUBJID')[['SUBJID', 'LSSLD_Change', 'Predicted_LSSLD_Change']]

In [343]:
predictions1.head()

,SUBJID,LSSLD_Change,Predicted_LSSLD_Change
0,3,0,0
1008,7,1,1
12320,9,1,1
18592,13,0,0
20176,15,1,1


In [344]:
predictions1.to_csv("C:\\Users\\sdsek\\OneDrive\\Desktop\\Kolate.AI\\1183\\Regression Model (Tumour Size,New Lesions)\\Output\\Predictions for Change in Tumour Size.csv", index=False)
print("Saved.")

Saved.
